# Heurística de consolidación

**Consideraciones**
- Cada camión límite de 8H
- Cada camión límite de 10T (10.000 kilos o cantidad)
- Atender a clientes premium primero, dps los otros pq tengo más tiempo
- Considerar que los camioneros paran??

Esto es diferente porque necesitamos las fechas de las ventas, pero se parece al trabajo de BDD de construccion_datos

In [1]:
import pandas as pd
import numpy as np
import construccion_datos
from modelo_V_consolidacion import LocalizacionOptima
import construccion_datos

# Importación base de datos
bdd_categoria = "BDD_Bodegas_Categorizada.xlsx"
bdd_ventas = pd.read_excel(bdd_categoria)
bdd_bodegas = pd.read_excel("BDD_Bodegas.xlsx", sheet_name=2)
bdd_comunas = pd.read_excel("BDD_Bodegas.xlsx", sheet_name=3)
bdd_comuna_bodega = pd.read_excel("distancias_comunas_bodegas.xlsx")

# Pasamos la columna de fechas a formato de fechas
bdd_ventas['Fecha'] = pd.to_datetime(bdd_ventas['Fecha'])

# Agrupación de ventas por cliente y completar comuna
bdd_ventas_2 = bdd_ventas.merge(bdd_comunas, left_on='Comuna Despacho', right_on='Comuna')
bdd_ventas_2['Cantidad'] = bdd_ventas_2['Cantidad'].apply(lambda x: 0.01 if x == 0 else x)
bdd_ventas_2['Nuevo_ID'] = range(len(bdd_ventas_2))


dict_bodegas = bdd_bodegas.set_index('ID Bodega')[['LAT', 'LONG']].to_dict(orient='index') 
dict_ventas = bdd_ventas_2.set_index('Nuevo_ID')[['ID Cliente', 'Cantidad', 'Comuna Despacho', 'LAT', 'LON', 'ID Bodega Despacho', 'Categoria', 'Fecha']].to_dict(orient='index') 

In [2]:
#dict_ventas = construccion_datos.dict_ventas #Necesito la fecha!
dict_bodegas = construccion_datos.dict_bodegas
bdd_ventas_agrupadas = construccion_datos.bdd_ventas_agrupadas

In [3]:
# Para esto ocupamos el dict ventas agrupadas, para tener solo los clientes y las distancias entre ellos
dict_ventas_agrupadas = bdd_ventas_agrupadas.set_index('ID Cliente')[['Cantidad', 'Comuna Despacho', 'LAT', 'LON', 'ID Bodega Despacho', 'Categoria']].to_dict(orient='index') 

Necesito las distancias Manhattan entre clientes. Además incluyo clientes que tenían demanda acumulada 0

In [4]:
#Necesito d_Manhattan cliente-cliente preprocesado
dict_ventas_b = dict_ventas_agrupadas.copy()
radio_tierra = 6371000  # metros
d_Manhattan_entre_clientes = dict() # Distancias Manhattan

for i in dict_ventas_b.keys():
    d_Manhattan_entre_clientes[i] = dict()
    # creamos una iteración sobre los id de bodegas
    for j in dict_ventas_b.keys():
        # Sacamos la diferencia y hacemos la conversión a metros
        lat_diff = abs(dict_ventas_b[j]['LAT'] - dict_ventas_b[i]['LAT']) * (np.pi / 180) * radio_tierra
        lon_diff = abs(dict_ventas_b[j]['LON'] - dict_ventas_b[i]['LON']) * (np.pi / 180) * radio_tierra * np.cos((dict_ventas_b[j]['LAT'] + dict_ventas_b[i]['LAT']) * 0.5 * (np.pi / 180)) 
        if round((lat_diff + lon_diff)/1000, 2) > 0:
            d_Manhattan_entre_clientes[i][j] = round((lat_diff + lon_diff)/1000, 2)  # Distancia Manhattan en kilometros, redondeado al segundo decimal
        elif round((lat_diff + lon_diff)/1000, 2) == 0:
            d_Manhattan_entre_clientes[i][j] = 0.01

In [5]:
#Necesito d_Manhattan cliente-bodega
d_Manhattan_bodega_cliente = construccion_datos.d_Manhattan

Corro modelo de optimización (se pueden cambiar los parámetros)

In [22]:
def solucion_p_median(p):
    # Crear una instancia de LocalizacionOptima
    modelo = LocalizacionOptima(p, construccion_datos.d_Manhattan)

    # Llamar al método que resuelve el modelo
    modelo.resolver()

    # Recuperar el diccionario solución
    diccionario_solucion = modelo.resultados

    return diccionario_solucion

diccionario_solucion = solucion_p_median(10)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 31384 rows, 28540 columns and 85600 nonzeros
Model fingerprint: 0xea46b597
Variable types: 28530 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 31384 rows and 28540 columns
Presolve time: 0.05s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.03 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 116401 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.164005100000e+05, best bound 1.164005100000e+05, gap 0.0000%


## Heurística

Hacemos diccionario que tiene fecha y la info de los clientes de esa fecha

In [23]:
# Clientes por fecha
# Supongamos que 'diccionario' es tu diccionario con 66,000 filas

clientes_por_fecha_2 = {}

for cliente_id, cliente_info in dict_ventas.items():
    fecha = cliente_info['Fecha']
    
    # Si la fecha ya está en el diccionario, agregamos el cliente a la lista existente
    if fecha in clientes_por_fecha_2:
        clientes_por_fecha_2[fecha].append(cliente_info)
    else:
        # Si la fecha no está en el diccionario, creamos una nueva lista con este cliente
        clientes_por_fecha_2[fecha] = [cliente_info]

Calculamos el valor objetivo de la solución previa sin consolidación

In [24]:
def calcular_valor_objetivo_previo(diccionario_solucion, clientes_por_fecha_2, ida_o_vuelta):
    valor_objetivo_previo = 0

    if ida_o_vuelta == "ida":
        for fecha in clientes_por_fecha_2:
            for cliente in clientes_por_fecha_2[fecha]:
                valor_objetivo_previo += diccionario_solucion[cliente["ID Cliente"]]["Tiempo"] 

    elif ida_o_vuelta == "vuelta":
        for fecha in clientes_por_fecha_2:
            for cliente in clientes_por_fecha_2[fecha]:
                valor_objetivo_previo += diccionario_solucion[cliente["ID Cliente"]]["Tiempo"]*2
    
    return valor_objetivo_previo

In [25]:
calcular_valor_objetivo_previo(solucion_p_median(10), clientes_por_fecha_2, "vuelta")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 31384 rows, 28540 columns and 85600 nonzeros
Model fingerprint: 0xea46b597
Variable types: 28530 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 31384 rows and 28540 columns
Presolve time: 0.04s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.03 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 116401 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.164005100000e+05, best bound 1.164005100000e+05, gap 0.0000%


119818.46133336831

Definimos los parámetros de la consolidación

In [26]:
# Es muy greedy, se puede mejorar

velocidad = 45
capacidad_maxima = 10000
tiempo_maximo = 8

In [27]:
# Función para verificar si cliente cabe en camión
def cabe_en_camion(camion, cliente):
    tiempo_a_cliente = d_Manhattan_entre_clientes[camion[2][-1]][cliente[1]] / velocidad  #aca podriamos hacer un 3-opt
    if camion[0] + cliente[0] <= capacidad_maxima and camion[1] + tiempo_a_cliente <= tiempo_maximo:
        return (True, tiempo_a_cliente)
    else:
        return (False, tiempo_a_cliente)

In [28]:
# Iteraciones
clientes_por_fecha = dict(clientes_por_fecha_2)

# La solución se guarda con diccionario por fecha
solucion_consolidacion = dict() 

Hacemos consolidación
- Revisa los clientes por fecha
- Primero revisa premium, luego gold y finalmente silver
- Por cada cliente, dentro de su categoria, revisa si cabe en un camion existente o crea uno nuevo (por capacidad y tiempo)

In [29]:
def consolidacion(clientes_por_fecha, diccionario_solucion):

    for fecha in clientes_por_fecha: 
        solucion_fecha = list()
        for cliente in clientes_por_fecha[fecha]:
            if cliente['Categoria'] == "Premium":
                
                # Si es que no es primer cliente
                if len(solucion_fecha) != 0:
                    cupo = False
                    for indice, camion in enumerate(solucion_fecha):

                        # Si cabe en algun camion, lo agrego
                        cliente_iteracion = [cliente["Cantidad"], cliente["ID Cliente"]]
                        cabe = cabe_en_camion(camion, cliente_iteracion)

                        if cabe[0] and diccionario_solucion[cliente["ID Cliente"]]["Bodega Asignada"] == camion[2][0]:
                            solucion_fecha[indice][0] += cliente["Cantidad"]
                            solucion_fecha[indice][1] += cabe[1]
                            solucion_fecha[indice][2].append(cliente["ID Cliente"])
                            cupo = True
                            break
                    
                    if not cupo:
                        tiempo = diccionario_solucion[cliente["ID Cliente"]]["Tiempo"]
                        solucion_fecha.append([cliente["Cantidad"], tiempo, [diccionario_solucion[cliente["ID Cliente"]]["Bodega Asignada"], cliente["ID Cliente"]] ])
                
                # Si es que es primer cliente
                else:
                    # Creo camión ############
                    tiempo = diccionario_solucion[cliente["ID Cliente"]]["Tiempo"]
                    solucion_fecha.append([cliente["Cantidad"], tiempo, [diccionario_solucion[cliente["ID Cliente"]]["Bodega Asignada"], cliente["ID Cliente"]] ])
        
        for cliente in clientes_por_fecha[fecha]:
            if cliente['Categoria'] == "Gold":
                # Si es que no es primer cliente

                if len(solucion_fecha) != 0:
                    cupo = False
                    for indice, camion in enumerate(solucion_fecha):

                        # Si cabe en algun camion, lo agrego
                        cliente_iteracion = [cliente["Cantidad"], cliente["ID Cliente"]]
                        cabe = cabe_en_camion(camion, cliente_iteracion)

                        if cabe[0] and diccionario_solucion[cliente["ID Cliente"]]["Bodega Asignada"] == camion[2][0]:
                            solucion_fecha[indice][0] += cliente["Cantidad"]
                            solucion_fecha[indice][1] += cabe[1]
                            solucion_fecha[indice][2].append(cliente["ID Cliente"])
                            
                            cupo = True
                            break
                    
                    if not cupo:
                        tiempo = diccionario_solucion[cliente["ID Cliente"]]["Tiempo"]
                        solucion_fecha.append([cliente["Cantidad"], tiempo, [diccionario_solucion[cliente["ID Cliente"]]["Bodega Asignada"], cliente["ID Cliente"]] ])
                
                # Si es que es primer cliente
                else:
                    # Creo camión ############
                    tiempo = diccionario_solucion[cliente["ID Cliente"]]["Tiempo"]
                    solucion_fecha.append([cliente["Cantidad"], tiempo, [diccionario_solucion[cliente["ID Cliente"]]["Bodega Asignada"], cliente["ID Cliente"]] ])

        for cliente in clientes_por_fecha[fecha]:
            if cliente['Categoria'] == "Silver":

                # Si es que no es primer cliente
                if len(solucion_fecha) != 0:
                    cupo = False
                    for indice, camion in enumerate(solucion_fecha):

                        # Si cabe en algun camion, lo agrego
                        cliente_iteracion = [cliente["Cantidad"], cliente["ID Cliente"]]
                        cabe = cabe_en_camion(camion, cliente_iteracion)

                        if cabe[0] and diccionario_solucion[cliente["ID Cliente"]]["Bodega Asignada"] == camion[2][0]:
                            solucion_fecha[indice][0] +=  cliente["Cantidad"]
                            solucion_fecha[indice][1] += cabe[1]
                            solucion_fecha[indice][2].append(cliente["ID Cliente"])
                            cupo = True
                            break
                    
                    if not cupo:
                        tiempo = diccionario_solucion[cliente["ID Cliente"]]["Tiempo"]
                        solucion_fecha.append([cliente["Cantidad"], tiempo, [diccionario_solucion[cliente["ID Cliente"]]["Bodega Asignada"], cliente["ID Cliente"]] ])
                
                # Si es que es primer cliente
                else:
                    # Creo camión ############
                    tiempo = diccionario_solucion[cliente["ID Cliente"]]["Tiempo"]
                    solucion_fecha.append([cliente["Cantidad"], tiempo, [diccionario_solucion[cliente["ID Cliente"]]["Bodega Asignada"], cliente["ID Cliente"]] ])
            
        solucion_consolidacion[fecha] = solucion_fecha   

    return solucion_consolidacion

Calculo solución consolidación y comparación

In [34]:
def calcular_valor_objetivo_consolidacion(solucion_consolidacion, ida_o_vuelta):
    valor_objetivo_actual = 0

    if ida_o_vuelta == "ida":
        for fecha in solucion_consolidacion:
            for camion in solucion_consolidacion[fecha]:
                valor_objetivo_actual += camion[1]
    
    elif ida_o_vuelta == "vuelta":
        for fecha in solucion_consolidacion:
            for camion in solucion_consolidacion[fecha]:
                valor_objetivo_actual += camion[1]
                valor_objetivo_actual += d_Manhattan_bodega_cliente[camion[2][-1]][camion[2][0]] / velocidad # Sacar si solo ida

    return valor_objetivo_actual

In [38]:
consolidacion_vs_previo = dict()

for cantidad_bodegas in [3,5,10]:
    diccionario_solucion = solucion_p_median(cantidad_bodegas)
    clientes_por_fecha = dict(clientes_por_fecha_2)
    solucion_consolidacion = consolidacion(clientes_por_fecha, diccionario_solucion)

    valor_objetivo_consolidacion_ida = calcular_valor_objetivo_consolidacion(solucion_consolidacion, "ida")
    valor_objetivo_previo_ida = calcular_valor_objetivo_previo(diccionario_solucion, clientes_por_fecha_2, "ida")
    valor_objetivo_consolidacion_vuelta = calcular_valor_objetivo_consolidacion(solucion_consolidacion, "vuelta")
    valor_objetivo_previo_vuelta = calcular_valor_objetivo_previo(diccionario_solucion, clientes_por_fecha_2, "vuelta")

    consolidacion_vs_previo[(cantidad_bodegas)] = [valor_objetivo_consolidacion_ida, valor_objetivo_previo_ida, valor_objetivo_consolidacion_vuelta, valor_objetivo_previo_vuelta]

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 31384 rows, 28540 columns and 85600 nonzeros
Model fingerprint: 0xa56e59af
Variable types: 28530 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 29007 rows and 26370 columns
Presolve time: 0.10s
Presolved: 2377 rows, 2170 columns, 6490 nonzeros
Variable types: 2160 continuous, 10 integer (10 binary)
Found heuristic solution: objective 784113.74000

Root relaxation: objective 3.474816e+05, 624 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*  

In [41]:
for valor in consolidacion_vs_previo:
    print("Cantidad bodegas: ", valor)
    print("Valor objetivo consolidacion ida: ", consolidacion_vs_previo[valor][0], "Valor objetivo previo:", consolidacion_vs_previo[valor][1], "diferencia porcentual:", 100*round((consolidacion_vs_previo[valor][0] - consolidacion_vs_previo[valor][1])/consolidacion_vs_previo[valor][1], 2))
    print("Valor objetivo consolidacion vuelta: ", consolidacion_vs_previo[valor][2], "Valor objetivo previo:", consolidacion_vs_previo[valor][3], "diferencia porcentual:", 100*round((consolidacion_vs_previo[valor][2] - consolidacion_vs_previo[valor][3])/consolidacion_vs_previo[valor][3], 2))
    print("\n")

Cantidad bodegas:  3
Valor objetivo consolidacion ida:  116676.07711111104 Valor objetivo previo: 162497.17355555695 diferencia porcentual: -28.000000000000004
Valor objetivo consolidacion vuelta:  183424.34288888043 Valor objetivo previo: 324994.3471111139 diferencia porcentual: -44.0


Cantidad bodegas:  5
Valor objetivo consolidacion ida:  82219.98755555536 Valor objetivo previo: 105761.99866669456 diferencia porcentual: -22.0
Valor objetivo consolidacion vuelta:  116090.74066666799 Valor objetivo previo: 211523.99733338912 diferencia porcentual: -45.0


Cantidad bodegas:  10
Valor objetivo consolidacion ida:  54894.36688888896 Valor objetivo previo: 59909.230666684154 diferencia porcentual: -8.0
Valor objetivo consolidacion vuelta:  74995.14222222334 Valor objetivo previo: 119818.46133336831 diferencia porcentual: -37.0




#### Comparar cantidad de camiones

In [45]:
def calcular_camiones(solucion_consolidacion):
    cantidad_camiones = 0

    for fecha in solucion_consolidacion:
        for camion in solucion_consolidacion[fecha]:
            cantidad_camiones += 1
    
    return cantidad_camiones

In [46]:
calcular_camiones(solucion_consolidacion)

16013

In [47]:
len(dict_ventas)

65891